In [1]:
import os
import sys
import random
from langchain_community.vectorstores import Chroma
from langchain_experimental.utilities import PythonREPL
from IPython.display import display, Markdown
import jsonlines
import chromadb
import yaml

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from enterprise_knowledge_retriever.src.document_retrieval import DocumentRetrieval
from utils.agents.static_RAG_with_coding import CodeRAG

examples = []

CONFIG_PATH = os.path.join(kit_dir, "config.yaml")

/Users/codym/CAP/aisk_forked/ai-starter-kit/comp_rag_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_config(filename: str) -> dict:
    try:
        with open(filename, "r") as file:
            return yaml.safe_load(file)
    except FileNotFoundError:
        raise FileNotFoundError(
            f"The YAML configuration file {filename} was not found."
        )
    except yaml.YAMLError as e:
        raise RuntimeError(f"Error parsing YAML file: {e}")

In [6]:
documentRetrieval = DocumentRetrieval()
*_, embedding_model_info, retrieval_info, _ = documentRetrieval.get_config_info()
embeddings = documentRetrieval.load_embedding_model()

2024-07-12 10:52:18,894 [INFO] - Load pretrained SentenceTransformer: intfloat/e5-large-v2


load INSTRUCTOR_Transformer


2024-07-12 10:52:20,921 [INFO] - Use pytorch device: cpu


max_seq_length  512


In [7]:
vectorstore = Chroma(
    persist_directory=kit_dir
    + "/data/uber_lyft.chromadb",  # Create for your own experiments.
    embedding_function=embeddings,
)

2024-07-12 10:52:20,942 [INFO] - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [8]:
# instantiate rag
rag = CodeRAG(
    configs=CONFIG_PATH,
    embeddings=embeddings,
    vectorstore=vectorstore,
    examples=examples,
)

# Initialize chains
rag.initialize()

# Build nodes
workflow = rag.create_rag_nodes()
print(workflow)

# Build graph
app = rag.build_rag_graph(workflow)

In [9]:
# rag.display_graph(app)  # Uncomment to display graph

In [10]:
def log_answers(question, output_file):
    try:
        response = rag.call_rag(app, question)
    except:
        response = {}
        response["answer"] = "Failed"

    display(Markdown("---Response---"))
    display(Markdown(response["answer"]))

    if not os.path.exists(kit_dir + "/outputs"):
        os.makedirs(kit_dir + "/outputs")
    output_dir = kit_dir + "/outputs"

    with jsonlines.open(output_dir + f"/{output_file}", "a") as writer:
        writer.write({"question": question, "response": response["answer"]})

In [11]:
questions = [
    "Could the trading price of lyft stock be volatile?",
    "What was the difference in revenue for Uber between 2020 and 2021?",
    "What was the change in revenue for Lyft for 2020 and 2021 as a percentage?",
    "Provide the business overviews for Uber and Lyft.",
    "What are the growth strategies for Lyft and Uber?",
    "How do the growth strategies differ between Lyft and Uber?",
]

In [12]:
for question in questions:
    log_answers(question, "lyft_stock_price_Llama3_70B_Instruct.jsonl")

---Initializing---
Could the trading price of lyft stock be volatile?
---Decision---
{'generate_or_example': 'answer_generation'}
---ROUTE ORIGINAL QUESTION---
---ROUTING QUESTION---
answer_generation
---ROUTE QUESTION TO LLM---
---DETERMINING ENTITIES---
entities:  ['lyft']
['lyft.pdf']
---ANSWERING QUESTION---
Could the trading price of lyft stock be volatile?
---RERANKING DOCUMENTS---
---NUM DOCUMENTS RETRIEVED---
3
As of December 31, 2021, there were six stockholders of record of our Class B common stock. All shares of Class B common stock are beneficially owned by either Logan Green or John Zimmer.

Dividend Policy

We have never paid cash dividends on our capital stock and we do not anticipate paying any cash dividends in the foreseeable future.

Stock Performance Graph

This performance graph shall not be deemed “filed” with the SEC for purposes of Section 18 of the Exchange Act or incorporated by reference into any filing of Lyft, Inc. under the Securities Act.

The graph below

---Response---

Yes, the trading price of Lyft stock could be volatile.

---Initializing---
What was the difference in revenue for Uber between 2020 and 2021?
---Decision---
{'generate_or_example': 'answer_generation'}
---ROUTE ORIGINAL QUESTION---
---ROUTING QUESTION---
subquery_generation
---ROUTE QUESTION TO SUBQUERY_GENERATION---
---SUBQUESTIONS---
• What was the revenue for Uber in 2020?
• What was the revenue for Uber in 2021?
---DETERMINING ENTITIES---
entities:  ['Uber']
['uber.pdf']
---ANSWERING SUBQUESTIONS---
• What was the revenue for Uber in 2020?
---RERANKING DOCUMENTS---
---NUM DOCUMENTS RETRIEVED---
3
<table><thead><th rowspan="4">(In millions, except percentages) Monthly Active Platform Consumers (“MAPCs”) 2) 3) Trips @) Gross Bookings )</th><th colspan="3" rowspan="3">2020 93 5,025</th><th>2021</th><th colspan="2">2020 to 2021 % Change</th><th rowspan="4">2020 to 2021 %</th><th rowspan="4">Change 53%</th></thead><thead><th></th><th colspan="3"></th><th rowspan="3">118 6,368 90,415</th><th>27</th><th>%</th><th></th><th></th></thead><thead><

2024-07-12 10:53:35,842 [WARNING] - Python REPL can execute arbitrary code. Use with caution.


...

---Response---

The growth strategies for Lyft and Uber include:

**Lyft:**
Unfortunately, I couldn't find specific growth strategies for Lyft. However, you can try searching for Lyft's company reports, investor presentations, news articles, research firm reports, and social media channels to find more information on their growth strategies.

**Uber:**

1. Growing supply and demand on the platform
2. Increasing existing platform users' activity
3. Expanding to new markets
4. Providing high-quality support to users
5. Expanding business and increasing market share
6. Competing with competitors
7. Developing new products and technologies
8. Acquiring or investing in complementary businesses
9. Penetrating suburban and rural areas
10. Reducing costs to compete with personal vehicle ownership and public transportation.

I apologize, but I cannot provide a comprehensive answer for Lyft's growth strategies due to a lack of information.

---Initializing---
How do the growth strategies differ between Lyft and Uber?
---Decision---
{'generate_or_example': 'answer_generation'}
---ROUTE ORIGINAL QUESTION---
---ROUTING QUESTION---
subquery_generation
---ROUTE QUESTION TO SUBQUERY_GENERATION---
---SUBQUESTIONS---
• What is the growth strategy of Lyft?
• What is the growth strategy of Uber?
---DETERMINING ENTITIES---
entities:  Lyft
['lyft.pdf']
---ANSWERING SUBQUESTIONS---
• What is the growth strategy of Lyft?
---RERANKING DOCUMENTS---
---NUM DOCUMENTS RETRIEVED---
3
Demand for our offerings is highly sensitive to the price of rides, the rates for time and distance driven, incentives paid to drivers and the fees we charge drivers. Many factors, including operating costs, legal and regulatory requirements or constraints and our current and future competitors’ pricing and marketing strategies including increased incentives for drivers, could significantly affect our pricing strategies. Certain of our competitors offer, or may 

---Response---

The growth strategies of Lyft and Uber differ in their focus and approach. Lyft's strategy is centered around optimizing its core ridesharing business by leveraging its technology platform, scale, and user network density to improve efficiency and develop new offerings. In contrast, Uber's strategy is more diversified, focusing on expanding into various businesses beyond ridesharing, such as autonomous vehicles, trucking, food delivery, and personal mobility devices, among others.